***Copyright 2020 The TensorFlow Authors.***

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Restrições de formato pela ética com o TensorFlow Lattice

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lattice/tutorials/shape_constraints_for_ethics"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/lattice/tutorials/shape_constraints_for_ethics.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/lattice/tutorials/shape_constraints_for_ethics.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/lattice/tutorials/shape_constraints_for_ethics.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Este tutorial demonstra como a biblioteca do TensorFlow Lattice (TFL) pode ser usada para treinar modelos que se comportam *de maneira responsável* e não violam determinadas suposições que são *éticas* ou *justas*. Em especial, vamos focar em usar as restrições de monotonicidade para evitar a *penalização injusta* de atributos específicos. Este tutorial inclui a demonstração dos experimentos do artigo [*Deontological Ethics By Monotonicity Shape Constraints*](https://arxiv.org/abs/2001.11990) (Ética deontológica por restrições de formato de monotonicidade), de Serena Wang e Maya Gupta, publicado na [AISTATS 2020](https://www.aistats.org/).

Vamos usar os estimadores predefinidos do TFL em datasets públicos, mas observe que o tutorial inteiro também pode ser realizado com modelos construídos a partir de camadas Keras do TFL.

Antes de continuar, confira se o runtime tem todos os pacotes necessários instalados (conforme importados nas células de código abaixo).

## Configuração

Instale o pacote do TF Lattice:

In [ ]:
#@test {"skip": true}
!pip install tensorflow-lattice tensorflow_decision_forests seaborn

Importe os pacotes necessários:

In [ ]:
import tensorflow as tf
import tensorflow_lattice as tfl
import tensorflow_decision_forests as tfdf

import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import sys
import tempfile
logging.disable(sys.maxsize)

Valores padrão usados neste tutorial:

In [ ]:
# List of learning rate hyperparameters to try.
# For a longer list of reasonable hyperparameters, try [0.001, 0.01, 0.1].
LEARNING_RATES = [0.01]
# Default number of training epochs and batch sizes.
NUM_EPOCHS = 1000
BATCH_SIZE = 1000
# Directory containing dataset files.
DATA_DIR = 'https://raw.githubusercontent.com/serenalwang/shape_constraints_for_ethics/master'

# 1º estudo de caso: admissões na faculdade de direito

Na primeira parte deste tutorial, vamos considerar um estudo de caso usando o dataset Law School Admissions do Law School Admissions Council (LSAC). Vamos treinar um classificador para prever se um estudante passará ou não no exame da ordem dos advogados usando duas características: a nota do LSAT (teste de admissão) e o GPA (média das notas) do universitário.

Suponha que a pontuação do classificador foi usada para determinar as admissões ou as bolsas da faculdade de direito. De acordo com as normas sociais baseadas em mérito, devemos esperar que os estudantes com uma nota mais alta do GPA e do LSAT recebam uma pontuação mais alta do classificador. No entanto, vamos observar que é fácil para os modelos violarem essas normas intuitivas e, às vezes, penalizar as pessoas por ter uma nota mais alta do GPA ou LSAT.

Para solucionar esse problema de *penalização injusta*, podemos impor restrições de monotonicidade para que um modelo nunca penalize uma nota mais alta do GPA ou LSAT, em igualdade de condições. Neste tutorial, vamos mostrar como impor essas restrições de monotonicidade usando o TFL.

## Carregue os dados da faculdade de direito

In [ ]:
# Load data file.
law_file_name = 'lsac.csv'
law_file_path = os.path.join(DATA_DIR, law_file_name)
raw_law_df = pd.read_csv(law_file_path, delimiter=',')

Faça o pré-processamento do dataset:

In [ ]:
# Define label column name.
LAW_LABEL = 'pass_bar'

In [ ]:
def preprocess_law_data(input_df):
  # Drop rows with where the label or features of interest are missing.
  output_df = input_df[~input_df[LAW_LABEL].isna() & ~input_df['ugpa'].isna() &
                       (input_df['ugpa'] > 0) & ~input_df['lsat'].isna()]
  return output_df


law_df = preprocess_law_data(raw_law_df)

### Divida os dados em datasets de treinamento/validação/teste

In [ ]:
def split_dataset(input_df, random_state=888):
  """Splits an input dataset into train, val, and test sets."""
  train_df, test_val_df = train_test_split(
      input_df, test_size=0.3, random_state=random_state)
  val_df, test_df = train_test_split(
      test_val_df, test_size=0.66, random_state=random_state)
  return train_df, val_df, test_df


law_train_df, law_val_df, law_test_df = split_dataset(law_df)

### Visualize a distribuição dos dados

Primeiro, vamos visualizar a distribuição dos dados. Vamos plotar as notas do GPA e LSAT para todos os estudantes que foram aprovados e reprovados no exame da ordem dos advogados.

In [ ]:
def plot_dataset_contour(input_df, title):
  plt.rcParams['font.family'] = ['serif']
  g = sns.jointplot(
      x='ugpa',
      y='lsat',
      data=input_df,
      kind='kde',
      xlim=[1.4, 4],
      ylim=[0, 50])
  g.plot_joint(plt.scatter, c='b', s=10, linewidth=1, marker='+')
  g.ax_joint.collections[0].set_alpha(0)
  g.set_axis_labels('Undergraduate GPA', 'LSAT score', fontsize=14)
  g.fig.suptitle(title, fontsize=14)
  # Adust plot so that the title fits.
  plt.subplots_adjust(top=0.9)
  plt.show()

In [ ]:
law_df_pos = law_df[law_df[LAW_LABEL] == 1]
plot_dataset_contour(
    law_df_pos, title='Distribution of students that passed the bar')

In [ ]:
law_df_neg = law_df[law_df[LAW_LABEL] == 0]
plot_dataset_contour(
    law_df_neg, title='Distribution of students that failed the bar')

## Treine o modelo linear calibrado para prever a aprovação no exame

Em seguida, vamos treinar um *modelo linear calibrado* do TFL para prever se um estudante passará no exame da ordem dos advogados. As duas características de entrada serão a nota do LSAT e o GPA do universitário, e o rótulo de treinamento será a aprovação ou não do estudante.

Primeiro, vamos treinar um modelo linear calibrado sem nenhuma restrição. Em seguida, vamos treinar um modelo linear calibrado com restrições de monotonicidade e observar a diferença na saída e exatidão do modelo.

### Funções helper para treinar um estimador linear calibrado do TFL

Estas funções serão usadas para o estudo de caso da faculdade de direito, bem como para o estudo de caso abaixo de inadimplência no cartão de crédito.

In [ ]:
def train_tfl_estimator(train_df, monotonicity, learning_rate, num_epochs,
                        batch_size, get_input_fn,
                        get_feature_columns_and_configs):
  """Trains a TFL calibrated linear estimator.

  Args:
    train_df: pandas dataframe containing training data.
    monotonicity: if 0, then no monotonicity constraints. If 1, then all
      features are constrained to be monotonically increasing.
    learning_rate: learning rate of Adam optimizer for gradient descent.
    num_epochs: number of training epochs.
    batch_size: batch size for each epoch. None means the batch size is the full
      dataset size.
    get_input_fn: function that returns the input_fn for a TF estimator.
    get_feature_columns_and_configs: function that returns TFL feature columns
      and configs.

  Returns:
    estimator: a trained TFL calibrated linear estimator.

  """
  feature_columns, feature_configs = get_feature_columns_and_configs(
      monotonicity)

  model_config = tfl.configs.CalibratedLinearConfig(
      feature_configs=feature_configs, use_bias=False)

  estimator = tfl.estimators.CannedClassifier(
      feature_columns=feature_columns,
      model_config=model_config,
      feature_analysis_input_fn=get_input_fn(input_df=train_df, num_epochs=1),
      optimizer=tf.keras.optimizers.legacy.Adam(learning_rate))

  estimator.train(
      input_fn=get_input_fn(
          input_df=train_df, num_epochs=num_epochs, batch_size=batch_size))
  return estimator


def optimize_learning_rates(
    train_df,
    val_df,
    test_df,
    monotonicity,
    learning_rates,
    num_epochs,
    batch_size,
    get_input_fn,
    get_feature_columns_and_configs,
):
  """Optimizes learning rates for TFL estimators.

  Args:
    train_df: pandas dataframe containing training data.
    val_df: pandas dataframe containing validation data.
    test_df: pandas dataframe containing test data.
    monotonicity: if 0, then no monotonicity constraints. If 1, then all
      features are constrained to be monotonically increasing.
    learning_rates: list of learning rates to try.
    num_epochs: number of training epochs.
    batch_size: batch size for each epoch. None means the batch size is the full
      dataset size.
    get_input_fn: function that returns the input_fn for a TF estimator.
    get_feature_columns_and_configs: function that returns TFL feature columns
      and configs.

  Returns:
    A single TFL estimator that achieved the best validation accuracy.
  """
  estimators = []
  train_accuracies = []
  val_accuracies = []
  test_accuracies = []
  for lr in learning_rates:
    estimator = train_tfl_estimator(
        train_df=train_df,
        monotonicity=monotonicity,
        learning_rate=lr,
        num_epochs=num_epochs,
        batch_size=batch_size,
        get_input_fn=get_input_fn,
        get_feature_columns_and_configs=get_feature_columns_and_configs)
    estimators.append(estimator)
    train_acc = estimator.evaluate(
        input_fn=get_input_fn(train_df, num_epochs=1))['accuracy']
    val_acc = estimator.evaluate(
        input_fn=get_input_fn(val_df, num_epochs=1))['accuracy']
    test_acc = estimator.evaluate(
        input_fn=get_input_fn(test_df, num_epochs=1))['accuracy']
    print('accuracies for learning rate %f: train: %f, val: %f, test: %f' %
          (lr, train_acc, val_acc, test_acc))
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    test_accuracies.append(test_acc)
  max_index = val_accuracies.index(max(val_accuracies))
  return estimators[max_index]

### Funções helper para configurar características do dataset da faculdade de direito

Estas funções helper são específicas ao estudo de caso da faculdade de direito.

In [ ]:
def get_input_fn_law(input_df, num_epochs, batch_size=None):
  """Gets TF input_fn for law school models."""
  return tf.compat.v1.estimator.inputs.pandas_input_fn(
      x=input_df[['ugpa', 'lsat']],
      y=input_df['pass_bar'],
      num_epochs=num_epochs,
      batch_size=batch_size or len(input_df),
      shuffle=False)


def get_feature_columns_and_configs_law(monotonicity):
  """Gets TFL feature configs for law school models."""
  feature_columns = [
      tf.feature_column.numeric_column('ugpa'),
      tf.feature_column.numeric_column('lsat'),
  ]
  feature_configs = [
      tfl.configs.FeatureConfig(
          name='ugpa',
          lattice_size=2,
          pwl_calibration_num_keypoints=20,
          monotonicity=monotonicity,
          pwl_calibration_always_monotonic=False),
      tfl.configs.FeatureConfig(
          name='lsat',
          lattice_size=2,
          pwl_calibration_num_keypoints=20,
          monotonicity=monotonicity,
          pwl_calibration_always_monotonic=False),
  ]
  return feature_columns, feature_configs

### Funções helper para visualizar saídas do modelo treinado

In [ ]:
def get_predicted_probabilities(estimator, input_df, get_input_fn):
  if isinstance(estimator, tf.estimator.Estimator):
    predictions = estimator.predict(
        input_fn=get_input_fn(input_df=input_df, num_epochs=1))
    return [prediction['probabilities'][1] for prediction in predictions]
  else:
    return estimator.predict(tfdf.keras.pd_dataframe_to_tf_dataset(input_df))


def plot_model_contour(estimator, input_df, num_keypoints=20):
  x = np.linspace(min(input_df['ugpa']), max(input_df['ugpa']), num_keypoints)
  y = np.linspace(min(input_df['lsat']), max(input_df['lsat']), num_keypoints)

  x_grid, y_grid = np.meshgrid(x, y)

  positions = np.vstack([x_grid.ravel(), y_grid.ravel()])
  plot_df = pd.DataFrame(positions.T, columns=['ugpa', 'lsat'])
  plot_df[LAW_LABEL] = np.ones(len(plot_df))
  predictions = get_predicted_probabilities(
      estimator=estimator, input_df=plot_df, get_input_fn=get_input_fn_law)
  grid_predictions = np.reshape(predictions, x_grid.shape)

  plt.rcParams['font.family'] = ['serif']
  plt.contour(
      x_grid,
      y_grid,
      grid_predictions,
      colors=('k',),
      levels=np.linspace(0, 1, 11))
  plt.contourf(
      x_grid,
      y_grid,
      grid_predictions,
      cmap=plt.cm.bone,
      levels=np.linspace(0, 1, 11))  # levels=np.linspace(0,1,8));
  plt.xticks(fontsize=20)
  plt.yticks(fontsize=20)

  cbar = plt.colorbar()
  cbar.ax.set_ylabel('Model score', fontsize=20)
  cbar.ax.tick_params(labelsize=20)

  plt.xlabel('Undergraduate GPA', fontsize=20)
  plt.ylabel('LSAT score', fontsize=20)

## Treine o modelo linear calibrado (não monotônico) sem restrições

In [ ]:
nomon_linear_estimator = optimize_learning_rates(
    train_df=law_train_df,
    val_df=law_val_df,
    test_df=law_test_df,
    monotonicity=0,
    learning_rates=LEARNING_RATES,
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    get_input_fn=get_input_fn_law,
    get_feature_columns_and_configs=get_feature_columns_and_configs_law)

In [ ]:
plot_model_contour(nomon_linear_estimator, input_df=law_df)

## Treine o modelo linear calibrado monotônico

In [ ]:
mon_linear_estimator = optimize_learning_rates(
    train_df=law_train_df,
    val_df=law_val_df,
    test_df=law_test_df,
    monotonicity=1,
    learning_rates=LEARNING_RATES,
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    get_input_fn=get_input_fn_law,
    get_feature_columns_and_configs=get_feature_columns_and_configs_law)

In [ ]:
plot_model_contour(mon_linear_estimator, input_df=law_df)

## Treine outros modelos sem restrições

Demonstramos que os modelos lineares calibrados do TFL podem ser treinados como monotônicos em ambas as notas do LSAT e do GPA sem sacrificar muito a exatidão.

No entanto, como o modelo linear calibrado se compara com outros tipos de modelos, como redes neurais profundas (DNNs) ou árvores impulsionadas por gradiente (GBTs)? As DNNs e GBTs têm saídas razoavelmente justas? Para tirar essa dúvida, em seguida, vamos treinar uma DNN e uma GBT sem restrições. Vamos observar se ambas conseguem violar a monotonicidade nas notas do LSAT e do GPA com facilidade.

### Treine um modelo de rede neural profunda (DNN) sem restrições

A arquitetura já foi otimizada para alcançar uma validação de alta exatidão.

In [ ]:
feature_names = ['ugpa', 'lsat']

dnn_estimator = tf.estimator.DNNClassifier(
    feature_columns=[
        tf.feature_column.numeric_column(feature) for feature in feature_names
    ],
    hidden_units=[100, 100],
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.008),
    activation_fn=tf.nn.relu)

dnn_estimator.train(
    input_fn=get_input_fn_law(
        law_train_df, batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS))
dnn_train_acc = dnn_estimator.evaluate(
    input_fn=get_input_fn_law(law_train_df, num_epochs=1))['accuracy']
dnn_val_acc = dnn_estimator.evaluate(
    input_fn=get_input_fn_law(law_val_df, num_epochs=1))['accuracy']
dnn_test_acc = dnn_estimator.evaluate(
    input_fn=get_input_fn_law(law_test_df, num_epochs=1))['accuracy']
print('accuracies for DNN: train: %f, val: %f, test: %f' %
      (dnn_train_acc, dnn_val_acc, dnn_test_acc))

In [ ]:
plot_model_contour(dnn_estimator, input_df=law_df)

### Treine um modelo de árvore impulsionada por gradiente (GBT) sem restrições

A estrutura da árvore já foi otimizada para alcançar uma validação de alta exatidão.

In [ ]:
law_train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    law_train_df, label='pass_bar')
law_test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    law_test_df, label='pass_bar')
law_val_ds = tfdf.keras.pd_dataframe_to_tf_dataset(law_val_df, label='pass_bar')

tree_model = tfdf.keras.GradientBoostedTreesModel(
    features=[tfdf.keras.FeatureUsage(name=name) for name in feature_names],
    exclude_non_specified_features=True,
    num_threads=1,
    num_trees=20,
    max_depth=4,
    growing_strategy='BEST_FIRST_GLOBAL',
    random_seed=42,
    temp_directory=tempfile.mkdtemp(),
)
tree_model.compile(metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')])
tree_model.fit(law_train_ds, validation_data=law_val_ds, verbose=0)

tree_train_acc = tree_model.evaluate(law_train_ds, verbose=0)[1]
tree_val_acc = tree_model.evaluate(law_val_ds, verbose=0)[1]
tree_test_acc = tree_model.evaluate(law_test_ds, verbose=0)[1]
print('accuracies for GBT: train: %f, val: %f, test: %f' %
      (tree_train_acc, tree_val_acc, tree_test_acc))

In [ ]:
plot_model_contour(tree_model, input_df=law_df)

# 2º estudo de caso: dívidas de cartão de crédito

O segundo estudo de caso que vamos considerar neste tutorial é a previsão da probabilidade de alguém ter uma dívida de cartão de crédito. Vamos usar o dataset Default of Credit Card Clients do repositório UCI. Esses dados foram coletados de 30 mil usuários de cartão de crédito taiwaneses e contêm um rótulo binário indicando se um usuário atrasou ou não um pagamento durante um período. As características incluem estado civil, gênero, educação e período de atraso do pagamento nas faturas existentes, para cada um dos meses entre abril e setembro de 2005.

Como fizemos com o primeiro estudo de caso, vamos ilustrar novamente usando as restrições de monotonicidade para evitar a *penalização injusta*: se o modelo fosse usado para determinar o escore de crédito de um usuário, poderia parecer injusto se as pessoas fossem penalizadas por pagar faturas com antecedência, em igualdade de condições. Por isso, aplicamos uma restrição de monotonicidade que impede o modelo de penalizar pagamentos antecipados.

## Carregue os dados das dívidas

In [ ]:
# Load data file.
credit_file_name = 'credit_default.csv'
credit_file_path = os.path.join(DATA_DIR, credit_file_name)
credit_df = pd.read_csv(credit_file_path, delimiter=',')

In [ ]:
# Define label column name.
CREDIT_LABEL = 'default'

### Divida os dados em datasets de treinamento/validação/teste

In [ ]:
credit_train_df, credit_val_df, credit_test_df = split_dataset(credit_df)

### Visualize a distribuição dos dados

Primeiro, vamos visualizar a distribuição dos dados. Vamos plotar a média e o erro padrão da taxa de dívida observada para pessoas com diferentes estados civis e status de reembolso. O status de reembolso representa o número de meses que alguém atrasou o pagamento do crédito (desde abril de 2005).

In [ ]:
def get_agg_data(df, x_col, y_col, bins=11):
  xbins = pd.cut(df[x_col], bins=bins)
  data = df[[x_col, y_col]].groupby(xbins).agg(['mean', 'sem'])
  return data


def plot_2d_means_credit(input_df, x_col, y_col, x_label, y_label):
  plt.rcParams['font.family'] = ['serif']
  _, ax = plt.subplots(nrows=1, ncols=1)
  plt.setp(ax.spines.values(), color='black', linewidth=1)
  ax.tick_params(
      direction='in', length=6, width=1, top=False, right=False, labelsize=18)
  df_single = get_agg_data(input_df[input_df['MARRIAGE'] == 1], x_col, y_col)
  df_married = get_agg_data(input_df[input_df['MARRIAGE'] == 2], x_col, y_col)
  ax.errorbar(
      df_single[(x_col, 'mean')],
      df_single[(y_col, 'mean')],
      xerr=df_single[(x_col, 'sem')],
      yerr=df_single[(y_col, 'sem')],
      color='orange',
      marker='s',
      capsize=3,
      capthick=1,
      label='Single',
      markersize=10,
      linestyle='')
  ax.errorbar(
      df_married[(x_col, 'mean')],
      df_married[(y_col, 'mean')],
      xerr=df_married[(x_col, 'sem')],
      yerr=df_married[(y_col, 'sem')],
      color='b',
      marker='^',
      capsize=3,
      capthick=1,
      label='Married',
      markersize=10,
      linestyle='')
  leg = ax.legend(loc='upper left', fontsize=18, frameon=True, numpoints=1)
  ax.set_xlabel(x_label, fontsize=18)
  ax.set_ylabel(y_label, fontsize=18)
  ax.set_ylim(0, 1.1)
  ax.set_xlim(-2, 8.5)
  ax.patch.set_facecolor('white')
  leg.get_frame().set_edgecolor('black')
  leg.get_frame().set_facecolor('white')
  leg.get_frame().set_linewidth(1)
  plt.show()

In [ ]:
plot_2d_means_credit(credit_train_df, 'PAY_0', 'default',
                     'Repayment Status (April)', 'Observed default rate')

## Treine o modelo linear calibrado para prever a taxa de dívida

Em seguida, vamos treinar um *modelo linear calibrado* do TFL para prever se uma pessoa atrasará ou não o pagamento do crédito. As duas características de entrada serão o estado civil da pessoa e quanto meses ela atrasou o pagamento do crédito em abril (status de reembolso). O rótulo de treinamento indicará se a pessoa atrasou ou não o pagamento.

Primeiro, vamos treinar um modelo linear calibrado sem nenhuma restrição. Em seguida, vamos treinar um modelo linear calibrado com restrições de monotonicidade e observar a diferença na saída e exatidão do modelo.

### Funções helper para a configuração de características do dataset de dívidas

Estas funções helper são específicas ao estudo de caso de dívidas de cartão de crédito.

In [ ]:
def get_input_fn_credit(input_df, num_epochs, batch_size=None):
  """Gets TF input_fn for credit default models."""
  return tf.compat.v1.estimator.inputs.pandas_input_fn(
      x=input_df[['MARRIAGE', 'PAY_0']],
      y=input_df['default'],
      num_epochs=num_epochs,
      batch_size=batch_size or len(input_df),
      shuffle=False)


def get_feature_columns_and_configs_credit(monotonicity):
  """Gets TFL feature configs for credit default models."""
  feature_columns = [
      tf.feature_column.numeric_column('MARRIAGE'),
      tf.feature_column.numeric_column('PAY_0'),
  ]
  feature_configs = [
      tfl.configs.FeatureConfig(
          name='MARRIAGE',
          lattice_size=2,
          pwl_calibration_num_keypoints=3,
          monotonicity=monotonicity,
          pwl_calibration_always_monotonic=False),
      tfl.configs.FeatureConfig(
          name='PAY_0',
          lattice_size=2,
          pwl_calibration_num_keypoints=10,
          monotonicity=monotonicity,
          pwl_calibration_always_monotonic=False),
  ]
  return feature_columns, feature_configs

### Funções helper para visualizar saídas do modelo treinado

In [ ]:
def plot_predictions_credit(input_df,
                            estimator,
                            x_col,
                            x_label='Repayment Status (April)',
                            y_label='Predicted default probability'):
  predictions = get_predicted_probabilities(
      estimator=estimator, input_df=input_df, get_input_fn=get_input_fn_credit)
  new_df = input_df.copy()
  new_df.loc[:, 'predictions'] = predictions
  plot_2d_means_credit(new_df, x_col, 'predictions', x_label, y_label)

## Treine o modelo linear calibrado (não monotônico) sem restrições

In [ ]:
nomon_linear_estimator = optimize_learning_rates(
    train_df=credit_train_df,
    val_df=credit_val_df,
    test_df=credit_test_df,
    monotonicity=0,
    learning_rates=LEARNING_RATES,
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    get_input_fn=get_input_fn_credit,
    get_feature_columns_and_configs=get_feature_columns_and_configs_credit)

In [ ]:
plot_predictions_credit(credit_train_df, nomon_linear_estimator, 'PAY_0')

## Treine o modelo linear calibrado monotônico

In [ ]:
mon_linear_estimator = optimize_learning_rates(
    train_df=credit_train_df,
    val_df=credit_val_df,
    test_df=credit_test_df,
    monotonicity=1,
    learning_rates=LEARNING_RATES,
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    get_input_fn=get_input_fn_credit,
    get_feature_columns_and_configs=get_feature_columns_and_configs_credit)

In [ ]:
plot_predictions_credit(credit_train_df, mon_linear_estimator, 'PAY_0')